# Setup

In [7]:
import numpy as np

In [47]:
import dtOmitted
import random_forest
#import decision_tree
import logistic_regression
from decision_tree import DecisionTree
from random_forest import RandomForest
from logistic_regression import stochastic_gradient_descent

# Data

In [22]:
filename = 'data/SPECTF.dat'
data = np.loadtxt(filename, delimiter=',')
X = data[:, 1:]
Y = np.array([data[:, 0]]).T

X is the data 2D array, Y is the corresponding responses 1D array

In [23]:
print("X shape: ", X.shape)
print("Y shape: ", Y.shape)

X shape:  (267, 44)
Y shape:  (267, 1)


In [32]:
print(X)
print("\n", Y[0:5])

[[ 59.  52.  70. ...,  74.  64.  67.]
 [ 72.  62.  69. ...,  71.  56.  58.]
 [ 71.  62.  70. ...,  41.  51.  46.]
 ..., 
 [ 75.  73.  72. ...,  75.  67.  71.]
 [ 59.  62.  72. ...,  76.  70.  70.]
 [ 64.  66.  68. ...,  64.  57.  54.]]

 [[ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]]


Both X and Y contain integer values

# Accuracy:

In [29]:
def accuracy_score(Y_true, Y_predict):
    res = 0
    for i in range(len(Y_true)):
        if Y_true[i] == Y_predict[i]:
            res = res+ 1
    return res / len(Y_true)

# Evaluate Random Forest & Logistic Regression

In [51]:
trials = 20
all_accuracies_random_forest = np.zeros(trials)
all_std_random_forest = np.zeros(trials)

all_accuracies_logistic_regression = np.zeros(trials)
all_std_logistic_regression = np.zeros(trials)

for cross validation:

In [52]:
folds = 4  
fold_part = int(np.floor(X.shape[0] / folds))

Evaluate the performance of Random Forest & Logistic Regression average over 20 trials of 4-fold cross validation

In [73]:
for trial in range(trials):
    index_array = np.arange(X.shape[0])
    np.random.seed(trial*500 +17)
    np.random.shuffle(index_array)
    accuracy_random_forest = np.zeros(folds)
    accuracy_logistic_regression = np.zeros(folds)
    
    for i in range(folds):
        row_idx_train = index_array[np.r_[0: i * fold_part - 0, (i + 1) * fold_part: X.shape[0]]]
        Xtrain = X[row_idx_train[:, ], :]
        Ytrain = Y[row_idx_train]

        row_idx_test = index_array[i * fold_part: (i + 1) * fold_part]
        Xtest = X[row_idx_test[:, ], :]
        Ytest = Y[row_idx_test]

        ####RANDOM FOREST####
         # train Random Forest           
        classifier = RandomForest(num_trees = 15, max_tree_depth =100, ratio_per_tree=0.50)  ##random forest
        classifier.fit(Xtrain, Ytrain)
        
        # output predictions on the remaining data
        Y_pred_random_forest, config = classifier.predict(Xtest)
        

        #output predictions on the remaining data
        Y_pred = classifier.predict(Xtest)
        accuracy_random_forest[i] = accuracy_score(Ytest, Y_pred_random_forest)
        
        ########
        
        ####Logistic Regression####
        ddd = np.column_stack((np.ones(np.array(Xtrain).shape[0]), np.array(Xtrain)))
        beta_hat = stochastic_gradient_descent(ddd, (np.array(Ytrain)).ravel(), epsilon=0.0001, l=1, step_size=0.05,max_steps=1000)

        dd = np.column_stack((np.ones(np.array(Xtest).shape[0]), np.array(Xtest)))

        Y_pred_logistic = []
        for j in range(dd.shape[0]):
            if dd[j].T.dot(beta_hat) > 0:
                Y_pred_logistic.append(1)
            else:
                Y_pred_logistic.append(0)

        accuracy_logistic_regression[i] = accuracy_score(np.array(Ytest.ravel()), np.array(Y_pred_logistic))
        ########
        
    #print(trial, ":  ","Random Forest: ",np.mean(accuracy_random_forest),", Logistic Regression: ", np.mean(accuracy_logistic_regression))
        
    all_accuracies_random_forest[trial] = np.mean(accuracy_random_forest)
    all_std_logistic_regression[trial] = np.mean(accuracy_logistic_regression)

In [74]:
meanRandomForestAccuracy = np.mean(all_accuracies_random_forest)
stddevRandomForestAccuracy = np.std(all_std_logistic_regression)

meanLogisticRegressionAccuracy = np.mean(all_std_logistic_regression)
stddevLogisticRegressionAccuracy = np.std(all_std_logistic_regression)

In [75]:
print("### Random Forest ###")
print("Accuracy: ", meanRandomForestAccuracy)
print("Standard Deviation: ", stddevRandomForestAccuracy)

### Random Forest ###
Accuracy:  0.746212121212
Standard Deviation:  1.11022302463e-16


##### Random Forest ###

Accuracy:  0.746212121212
    
Standard Deviation:  1.11022302463e-16

In [76]:
print("### Logistic Regression ###")
print("Accuracy: ", meanLogisticRegressionAccuracy)
print("Standard Deviation: ", stddevLogisticRegressionAccuracy)

### Logistic Regression ###
Accuracy:  0.651515151515
Standard Deviation:  1.11022302463e-16


##### Logistic Regression ###
Accuracy:  0.651515151515
    
Standard Deviation:  1.11022302463e-16

# Evaluate Decision Tree

In [34]:
trials = 100
all_accuracies = np.zeros(trials)

for cross validation:

In [36]:
folds = 10  
fold_part = int(np.floor(X.shape[0] / folds))

Evaluate the performance of decision trees
average over 100 trials of 10-fold cross validation

In [43]:
for trial in range(trials):
    index_array = np.arange(X.shape[0])
    np.random.shuffle(index_array)
    accuracy = np.zeros(folds)
    stdd = np.zeros(folds)
    
    for i in range(folds):
        row_idx_train = index_array[np.r_[0: i * fold_part - 0, (i + 1) * fold_part: X.shape[0]]]
        Xtrain = X[row_idx_train[:, ], :]
        Ytrain = Y[row_idx_train]

        row_idx_test = index_array[i * fold_part: (i + 1) * fold_part]
        Xtest = X[row_idx_test[:, ], :]
        Ytest = Y[row_idx_test]

        # train the decision tree
        classifier = DecisionTree(100)    
        classifier.fit(Xtrain, Ytrain)
        
        # presdict on test set
        Y_pred = classifier.predict(Xtest)     
        
        #output predictions on the remaining data
        Y_pred = classifier.predict(Xtest)
        accuracy[i] = accuracy_score(Ytest, Y_pred)
        
    all_accuracies[trial] = np.mean(accuracy)

In [44]:
meanDecisionTreeAccuracy = np.mean(all_accuracies)
stddevDecisionTreeAccuracy = np.std(all_accuracies)

In [45]:
print(meanDecisionTreeAccuracy)
print(stddevDecisionTreeAccuracy)

0.737653846154
0.018781639971


#### Decision Tree
Accuracy: 0.737653846154

Standard Deviation: 0.018781639971